In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as spr
from scipy.optimize import linprog
from numba import njit
import gurobipy as grb
import time


data_X = pd.read_csv("https://github.com/math-econ-code/mec_optim_2021-01/raw/master/data_mec_optim/marriage_personality-traits/Xvals.csv")
data_Y = pd.read_csv("https://github.com/math-econ-code/mec_optim_2021-01/raw/master/data_mec_optim/marriage_personality-traits/Yvals.csv")
data_affmat = pd.read_csv("https://github.com/math-econ-code/mec_optim_2021-01/raw/master/data_mec_optim/marriage_personality-traits/affinitymatrix.csv")

In [2]:
nobs = 1158

sdX = data_X.std().to_numpy()
sdY = data_Y.std().to_numpy()
mX = data_X.mean().to_numpy()
mY = data_Y.mean().to_numpy()

affmat = data_affmat.to_numpy()[0: 10, 1:]
Xvals = ((data_X-mX)/sdX).to_numpy()[0:nobs, :]
Yvals = ((data_Y-mY)/sdY).to_numpy()[0:nobs, :]
print('Xvals shape:', Xvals.shape)
print('Yvals shape:', Yvals.shape)
print('Affinity Matrix shape:', affmat.shape)

Xvals shape: (1158, 10)
Yvals shape: (1158, 10)
Affinity Matrix shape: (10, 10)


In [3]:
class random_panel:
    def __init__(self, init_matrix, time=20):
        self.time = time
        self.init_matrix = init_matrix
    def generate(self):
        X_panel = np.empty((self.time, self.init_matrix.shape[0], self.init_matrix.shape[1]))
        X_panel[0, :, :] = self.init_matrix
        for i in range(self.time - 1):
            X_panel[i+1, :, :] = X_panel[i, :, :] + np.random.randn(self.init_matrix.shape[0], self.init_matrix.shape[1])
        return X_panel

In [4]:
n = Xvals.shape[0]
t = 15

In [5]:
Xvals_dyna = random_panel(Xvals, t).generate()
Yvals_dyna = random_panel(Yvals.T, t).generate()

print(Xvals_dyna.shape)
print(Yvals_dyna.shape)

phi_dyna = Xvals_dyna @ affmat @ Yvals_dyna
print(phi_dyna.shape)
vecphi = phi_dyna.flatten()

(15, 1158, 10)
(15, 10, 1158)
(15, 1158, 1158)


In [6]:
p = np.ones((t, n, 1))/n
q = np.ones((t, n, 1))/n
d = np.concatenate((p,q), axis = None)

sparse_one = spr.csr_matrix(np.ones(n).reshape(1, n))

A = spr.kron(spr.identity(n*t), np.ones(n).reshape(1, n))
B = spr.kron(spr.kron(spr.identity(t), sparse_one), spr.identity(n))

Aconstr = spr.vstack([A, B])

In [7]:
m=grb.Model('Optimal Marriage')
x = m.addMVar(shape=t * n**2, name="x")
m.setObjective(vecphi @ x, grb.GRB.MAXIMIZE)
m.addConstr(Aconstr @ x == d, name="Constr")
m.optimize()

Academic license - for non-commercial use only - expires 2021-07-30
Using license file c:\gurobi911\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 34740 rows, 20114460 columns and 40228920 nonzeros
Model fingerprint: 0xd1045040
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-08, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-04, 9e-04]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 0 rows and 0 columns (presolve time = 15s) ...
Presolve removed 0 rows and 0 columns (presolve time = 21s) ...
Presolve removed 0 rows and 0 columns (presolve time = 29s) ...
Presolve removed 0 rows and 0 columns (presolve time = 60s) ...
Presolve removed 0 rows and 0 columns (presolve time = 60s) ...
Presolve removed 0 rows and 0 columns (presolve time = 65s) ...
Presolve rem

In [8]:
pi_panel = np.array(m.getAttr('x')).reshape(t, n, n)

In [9]:
def matching_of_man(man):
    for time in range(t):
        print('Period', time+1, ': Woman', np.argwhere(pi_panel[time, man-1,:] != 0)[0][0] + 1)

In [10]:
matching_of_man(1)

Period 1 : Woman 576
Period 2 : Woman 502
Period 3 : Woman 247
Period 4 : Woman 261
Period 5 : Woman 1009
Period 6 : Woman 53
Period 7 : Woman 404
Period 8 : Woman 562
Period 9 : Woman 562
Period 10 : Woman 412
Period 11 : Woman 53
Period 12 : Woman 261
Period 13 : Woman 788
Period 14 : Woman 801
Period 15 : Woman 829
